## Example that trains two Naive Bayes classifiers and explore some of the resulting model
#### Code adapted from:
http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html#sphx-glr-auto-examples-text-document-classification-20newsgroups-py

In [3]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause

#from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.utils.extmath import density
from sklearn import metrics


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


# parse commandline arguments
op = OptionParser()
op.add_option("--report",
              action="store_true", dest="print_report",
              help="Print a detailed classification report.")
op.add_option("--chi2_select",
              action="store", type="int", dest="select_chi2",
              help="Select some number of features using a chi-squared test")
op.add_option("--confusion_matrix",
              action="store_true", dest="print_cm",
              help="Print the confusion matrix.")
op.add_option("--top10",
              action="store_true", dest="print_top10",
              help="Print ten most discriminative terms per class"
                   " for every classifier.")
op.add_option("--all_categories",
              action="store_true", dest="all_categories",
              help="Whether to use all categories or not.")
op.add_option("--use_hashing",
              action="store_false",
              help="Use a hashing vectorizer.")
op.add_option("--n_features",
              action="store", type=int, default=10**3,
              help="n_features when using the hashing vectorizer.")
op.add_option("--filtered",
              action="store_true",
              help="Remove newsgroup information that is easily overfit: "
                   "headers, signatures, and quoting.")


def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)

print(__doc__)
op.print_help()
print()


# #############################################################################
# Load some categories from the training set
if opts.all_categories:
    categories = None
else:
    categories = [
        'alt.atheism',
        'talk.religion.misc',
        'comp.graphics',
        'sci.space',
    ]

if opts.filtered:
    remove = ('headers', 'footers', 'quotes')
else:
    remove = ()

print("Loading 20 newsgroups dataset for categories:")
print(categories if categories else "all")

data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)
print('data loaded')

# order of labels in `target_names` can be different from `categories`
target_names = data_train.target_names


def size_mb(docs):
    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

data_train_size_mb = size_mb(data_train.data)
data_test_size_mb = size_mb(data_test.data)

print("%d documents - %0.3fMB (training set)" % (
    len(data_train.data), data_train_size_mb))
print("%d documents - %0.3fMB (test set)" % (
    len(data_test.data), data_test_size_mb))
print("%d categories" % len(categories))
print()

# split a training set and a test set
y_train, y_test = data_train.target, data_test.target

print("Extracting features from the training data using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False,
                                   n_features=opts.n_features)
    X_train = vectorizer.transform(data_train.data)
else:
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english', max_features=1000)
    X_train = vectorizer.fit_transform(data_train.data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_train.shape)
print()

print("Extracting features from the test data using the same vectorizer")
t0 = time()
X_test = vectorizer.transform(data_test.data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_test_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_test.shape)
print()

# mapping from integer feature name to original token string
if opts.use_hashing:
    feature_names = None
else:
    feature_names = vectorizer.get_feature_names()

if opts.select_chi2:
    print("Extracting %d best features by a chi-squared test" %
          opts.select_chi2)
    t0 = time()
    ch2 = SelectKBest(chi2, k=opts.select_chi2)
    X_train = ch2.fit_transform(X_train, y_train)
    X_test = ch2.transform(X_test)
    if feature_names:
        # keep selected feature names
        feature_names = [feature_names[i] for i
                         in ch2.get_support(indices=True)]
    print("done in %fs" % (time() - t0))
    print()

if feature_names:
    feature_names = np.asarray(feature_names)


def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if opts.print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(target_names):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join(feature_names[top10]))))
        print()

    if opts.print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if opts.print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time


results = []

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))

Automatically created module for IPython interactive environment
Usage: __main__.py [options]

Options:
  -h, --help            show this help message and exit
  --report              Print a detailed classification report.
  --chi2_select=SELECT_CHI2
                        Select some number of features using a chi-squared
                        test
  --confusion_matrix    Print the confusion matrix.
  --top10               Print ten most discriminative terms per class for
                        every classifier.
  --all_categories      Whether to use all categories or not.
  --use_hashing         Use a hashing vectorizer.
  --n_features=N_FEATURES
                        n_features when using the hashing vectorizer.
  --filtered            Remove newsgroup information that is easily overfit:
                        headers, signatures, and quoting.
()
Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
data loaded
2034

In [4]:
mNB = MultinomialNB()
mNB.fit(X_train, y_train)
preds_score = mNB.predict(X_test)
preds_prob = mNB.predict_proba(X_test)

### Explore Documents

In [5]:
doc_id=22
print("MESSAGE %d\n===========\n\n" % doc_id)
print(data_test.data[doc_id])
print("===========\n")
print ("PREDICTED CLASS: %d" % y_test[doc_id])
print ("ACTUAL CLASS: %d\n" % preds_score[doc_id])

MESSAGE 22


From: mikec@sail.LABS.TEK.COM (Micheal Cranford)
Subject: the nature of light
Organization: Tektronix, Inc., Beaverton,  OR.
Lines: 21

In article <30185@ursa.bear.com> halat@pooh.bears (Jim Halat) writes:
[ deleted ]
>Take light as another example.  There are two theories: particle and
>wave.  Each one fails to predict the behavior of light as some point.
>So which is it: particle or wave?
[ deleted ]

  Your information on this topic is very much out of date.  Quantum Electro-
dynamics (QED - which considers light to be particles) has been experimentally
verified to about 14 decimal digits of precision under ALL tested conditions.
I'm afraid that this case, at least in the physics community, has been decided.
Laymen should consult "QED - The Strange Theory of Light and Matter" by Richard
P. Feynman and for the more technically minded there's "The Feynman Lectures on
Physics" by Feynman, Leighton and Sands (an excellent 3 volumes).  Case closed.


  UUCP:  uunet!tektronix

### Generate words from a class. P(x | y = k)

In [10]:
k=1


print("CLASS: %s" % target_names[k])
feature_prob = np.exp(mNB.feature_log_prob_)
print("Number of features per class: %d\n\n" % feature_prob.shape[1])

topN = 20
print("Top %d words for class %d:" % (topN, k))
for i in np.argsort(-feature_prob[k,:])[:topN]:
    print '\t', feature_names[i], '(%.0e)' % feature_prob[k,i]
    
print('\n\n')
#print(feature_prob[1].flatten().shape)
print("Words generated from the class conditional distribution:")
n_words =  25
idx = np.random.choice(feature_prob.shape[1], size=n_words, replace=True, p=feature_prob[k].flatten())
for i in xrange(idx.size):
    print '\t', feature_names[idx[i]], '(%.0e)' % feature_prob[k,idx[i]]

CLASS: comp.graphics
Number of features per class: 1000


Top 20 words for class 1:
	graphics (9e-03)
	thanks (6e-03)
	university (6e-03)
	posting (6e-03)
	host (5e-03)
	nntp (5e-03)
	com (5e-03)
	files (5e-03)
	image (5e-03)
	file (5e-03)
	help (5e-03)
	need (4e-03)
	looking (4e-03)
	program (4e-03)
	3d (4e-03)
	know (4e-03)
	computer (4e-03)
	windows (4e-03)
	does (4e-03)
	mail (4e-03)



Words generated from the class conditional distribution:
	looking (4e-03)
	polygon (3e-03)
	file (5e-03)
	australia (1e-03)
	stuff (1e-03)
	actually (1e-03)
	don (3e-03)
	programs (2e-03)
	ideas (1e-03)
	30 (1e-03)
	35 (8e-04)
	interface (1e-03)
	vehicle (2e-04)
	code (4e-03)
	issue (6e-04)
	windows (4e-03)
	week (1e-03)
	running (2e-03)
	image (5e-03)
	reasonable (6e-04)
	saw (7e-04)
	canada (1e-03)
	shuttle (3e-04)
	solntze (2e-04)
	netcom (8e-04)


### Words probabilities under different classes

In [11]:
word_to_id = {}
for i in xrange(feature_names.size):
    word_to_id[feature_names[i]] = i

In [13]:
word = 'vga'


word_id = word_to_id[word]

feature_prob = np.exp(mNB.feature_log_prob_)


idx = np.argsort(-feature_prob[:,word_id])

print('Category\t\tProb.')
print('========\t\t=====')
for cat in idx:
    print '{:<20}'.format(target_names[cat]), "\t%.1e" % feature_prob[cat,word_id]

Category		Prob.
========		=====
comp.graphics        	3.2e-03
talk.religion.misc   	3.0e-04
alt.atheism          	2.5e-04
sci.space            	2.2e-04
